# Explorando datos mundiales de COVID-19

In [1]:
import numpy as np
import pandas as pd

## Preparación de datos

1. Explore el archivo covid19_confirmados.csv y configure la función read_csv de pandas para importarlo como un DataFrame.
    - Utilice el nombre de país como índice principal
    - Elimine las columnas Lat y Long
    - Use TimeStamps para los "nombres de columna" asociados a fechas
    - Reduzca la columna Province/State, el DataFrame resultante debe tener los totales a nivel país

### Datos de casos confirmados

Primero, importamos los datos de casos confirmados a un dataframe

In [2]:
df_confirmados = pd.read_csv('./data/covid19_confirmados.csv')
df_confirmados.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/16/21,4/17/21,4/18/21,4/19/21,4/20/21,4/21/21,4/22/21,4/23/21,4/24/21,4/25/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,57612,57721,57793,57898,58037,58214,58312,58542,58730,58843
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,129307,129456,129594,129694,129842,129980,130114,130270,130409,130537
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,119323,119486,119642,119805,119992,120174,120363,120562,120736,120922
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,12712,12771,12805,12805,12874,12917,12942,13007,13024,13060
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,24122,24300,24389,24518,24661,24883,25051,25279,25492,25609


Las primeras cuatro columnas identifican el lugar y las 400 columnas siguientes corresponden al número de casos en ese lugar, durante 400 días consecutivos.

Algunos países tienen solo una fila, otros tienen varias, cada una correspondiente a una provincia o estado.

Luego, reduciremos la columna 'Province/State' y dejaremos una fila por país. Para facilitar esa reducción, crearemos un MultiIndex con las columnas 'Country/Region' y 'Province/State'.

In [3]:
df_confirmados.set_index(keys=['Country/Region', 'Province/State'], drop=True,inplace=True)

In [4]:
df_confirmados.head(5)

,,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,4/16/21,4/17/21,4/18/21,4/19/21,4/20/21,4/21/21,4/22/21,4/23/21,4/24/21,4/25/21
Country/Region,Province/State,,,,,,,,,,,,,,,,,,,,,
Afghanistan,NaN,33.93911,67.709953,0,0,0,0,0,0,0,0,...,57612,57721,57793,57898,58037,58214,58312,58542,58730,58843
Albania,NaN,41.15330,20.168300,0,0,0,0,0,0,0,0,...,129307,129456,129594,129694,129842,129980,130114,130270,130409,130537
Algeria,NaN,28.03390,1.659600,0,0,0,0,0,0,0,0,...,119323,119486,119642,119805,119992,120174,120363,120562,120736,120922
Andorra,NaN,42.50630,1.521800,0,0,0,0,0,0,0,0,...,12712,12771,12805,12805,12874,12917,12942,13007,13024,13060
Angola,NaN,-11.20270,17.873900,0,0,0,0,0,0,0,0,...,24122,24300,24389,24518,24661,24883,25051,25279,25492,25609


Pero antes de hacer esa reducción, ordenaremos un poco más el dataframe.

Eliminamos las columnas "Lat" y "Long":

In [5]:
df_confirmados.drop(labels=['Lat', 'Long'], axis=1, inplace=True)

In [6]:
df_confirmados.head(5)

,,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,4/16/21,4/17/21,4/18/21,4/19/21,4/20/21,4/21/21,4/22/21,4/23/21,4/24/21,4/25/21
Country/Region,Province/State,,,,,,,,,,,,,,,,,,,,,
Afghanistan,NaN,0,0,0,0,0,0,0,0,0,0,...,57612,57721,57793,57898,58037,58214,58312,58542,58730,58843
Albania,NaN,0,0,0,0,0,0,0,0,0,0,...,129307,129456,129594,129694,129842,129980,130114,130270,130409,130537
Algeria,NaN,0,0,0,0,0,0,0,0,0,0,...,119323,119486,119642,119805,119992,120174,120363,120562,120736,120922
Andorra,NaN,0,0,0,0,0,0,0,0,0,0,...,12712,12771,12805,12805,12874,12917,12942,13007,13024,13060
Angola,NaN,0,0,0,0,0,0,0,0,0,0,...,24122,24300,24389,24518,24661,24883,25051,25279,25492,25609


Luego, reinterpretamos las etiquetas de fecha como Timestamps en lugar de simples objetos.

Para eso, le aplicamos a cada columna la función `pd.to_datetime`.

In [7]:
df_confirmados.rename(mapper=pd.to_datetime, axis='columns', inplace=True);
df_confirmados.head(5)

,,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2021-04-16,2021-04-17,2021-04-18,2021-04-19,2021-04-20,2021-04-21,2021-04-22,2021-04-23,2021-04-24,2021-04-25
Country/Region,Province/State,,,,,,,,,,,,,,,,,,,,,
Afghanistan,NaN,0,0,0,0,0,0,0,0,0,0,...,57612,57721,57793,57898,58037,58214,58312,58542,58730,58843
Albania,NaN,0,0,0,0,0,0,0,0,0,0,...,129307,129456,129594,129694,129842,129980,130114,130270,130409,130537
Algeria,NaN,0,0,0,0,0,0,0,0,0,0,...,119323,119486,119642,119805,119992,120174,120363,120562,120736,120922
Andorra,NaN,0,0,0,0,0,0,0,0,0,0,...,12712,12771,12805,12805,12874,12917,12942,13007,13024,13060
Angola,NaN,0,0,0,0,0,0,0,0,0,0,...,24122,24300,24389,24518,24661,24883,25051,25279,25492,25609


Ahora sí reducimos el índice Province/State, para que quede solo una fila por país. Los valores para cada día serán la suma de casos dentro de las provincias para aquellos países en que habían varias provincias. Para los países que solo tenían una fila, el valor se mantiene. **TO DO**: Revisar que estas operaciones hayan quedado bien calculadas para ambos casos.

In [8]:
df_confirmados = df_confirmados.groupby(by='Country/Region').aggregate(np.sum)

In [9]:
df_confirmados.head(5)

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2021-04-16,2021-04-17,2021-04-18,2021-04-19,2021-04-20,2021-04-21,2021-04-22,2021-04-23,2021-04-24,2021-04-25
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,57612,57721,57793,57898,58037,58214,58312,58542,58730,58843
Albania,0,0,0,0,0,0,0,0,0,0,...,129307,129456,129594,129694,129842,129980,130114,130270,130409,130537
Algeria,0,0,0,0,0,0,0,0,0,0,...,119323,119486,119642,119805,119992,120174,120363,120562,120736,120922
Andorra,0,0,0,0,0,0,0,0,0,0,...,12712,12771,12805,12805,12874,12917,12942,13007,13024,13060
Angola,0,0,0,0,0,0,0,0,0,0,...,24122,24300,24389,24518,24661,24883,25051,25279,25492,25609


### Datos de población por país

Importamos los datos, usando el país como índice principal.

Solo importamos los datos de nombre de país y población. No usaremos los otros.

In [10]:
df_poblacion = pd.read_csv('./data/population_by_country_2020.csv',usecols=['Country (or dependency)','Population (2020)'],index_col='Country (or dependency)')

df_poblacion.head(5)

,Population (2020)
Country (or dependency),
China,1440297825
India,1382345085
United States,331341050
Indonesia,274021604
Pakistan,221612785


Ordenamos los países por orden alfabético.

In [11]:
df_poblacion.sort_index(inplace=True)
df_poblacion.head(5)

,Population (2020)
Country (or dependency),
Afghanistan,39074280
Albania,2877239
Algeria,43984569
American Samoa,55169
Andorra,77287


Debemos encontrar las diferencias entre las etiquetas de los índices de los dataframes.

Primero, veamos los tamaños de ambos índices.

In [12]:
print('n° filas df confirmados:', len(df_confirmados.index))
print('n° filas df población:', len(df_poblacion.index))
print('diferencia:', abs(len(df_confirmados.index) - len(df_poblacion.index)))

n° filas df confirmados: 192
n° filas df población: 235
diferencia: 43


Ahora, veamos qué etiquetas están en el dataframe de confirmados pero no en el de población.

In [13]:
df_confirmados.index.difference(df_poblacion.index)

Index(['Burma', 'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Cote d'Ivoire',
       'Czechia', 'Diamond Princess', 'Korea, South', 'Kosovo', 'MS Zaandam',
       'Saint Kitts and Nevis', 'Saint Vincent and the Grenadines',
       'Sao Tome and Principe', 'Taiwan*', 'US', 'West Bank and Gaza'],
      dtype='object')

In [14]:
len(_)

15

Ahora lo contrario, es decir, cuáles están en población pero no en confirmados.

In [15]:
df_poblacion.index.difference(df_confirmados.index)

Index(['American Samoa', 'Anguilla', 'Aruba', 'Bermuda',
       'British Virgin Islands', 'Caribbean Netherlands', 'Cayman Islands',
       'Channel Islands', 'Congo', 'Cook Islands', 'Curaçao',
       'Czech Republic (Czechia)', 'Côte d'Ivoire', 'DR Congo',
       'Faeroe Islands', 'Falkland Islands', 'French Guiana',
       'French Polynesia', 'Gibraltar', 'Greenland', 'Guadeloupe', 'Guam',
       'Hong Kong', 'Isle of Man', 'Kiribati', 'Macao', 'Martinique',
       'Mayotte', 'Montserrat', 'Myanmar', 'Nauru', 'New Caledonia', 'Niue',
       'North Korea', 'Northern Mariana Islands', 'Palau', 'Puerto Rico',
       'Réunion', 'Saint Barthelemy', 'Saint Helena', 'Saint Kitts & Nevis',
       'Saint Martin', 'Saint Pierre & Miquelon', 'Sao Tome & Principe',
       'Sint Maarten', 'South Korea', 'St. Vincent & Grenadines',
       'State of Palestine', 'Taiwan', 'Tokelau', 'Tonga', 'Turkmenistan',
       'Turks and Caicos', 'Tuvalu', 'U.S. Virgin Islands', 'United States',
       'Wallis 

In [16]:
len(_)

58

Hay varias diferencias entre los países incluidos en los dataframes. Por ejemplo: en un dataframe se usa el caracter '&' y en otros la palabra 'and' para nombrar a ciertos países. Además, hay diferencias semánticas y políticas en lo que se considera en la columna de los países.

En base a estas diferencias implementamos un procedimiento ad hoc para hacer que los países de ambos dataframes coincidan.

In [17]:
# Cambio de algunos símbolos
symbolChanges = [
    ['&', 'and'],
    ['St.', 'Saint'],
    ['ô', 'o']
]
for change in symbolChanges:
    df_poblacion.index = [country.replace(change[0], change[1]) for country in df_poblacion.index]


# Cambiamos valores específicos
fullChanges = [
    ['US','United States'],
    ['Burma','Myanmar'],
    ['Korea, South','South Korea'],
    ['Taiwan*','Taiwan'],
    ['Congo (Kinshasa)','DR Congo'],
    ['Congo (Brazzaville)','Congo'],
    ['Czechia','Czech Republic (Czechia)'],
    ['Saint Vincent and the Grenadines','Saint Vincent and Grenadines']
]

for change in fullChanges:  
    df_poblacion.index = [change[0] if country == change[1] else country for country in df_poblacion.index]


# Incluimos sectores dentro de otros ( sumando población )
includeChanges = [
    ['France','Wallis and Futuna'],
    ['France','New Caledonia'],
    ['France','French Guiana'],
    ['France','French Polynesia'],
    ['France','Guadeloupe'],
    ['France','Martinique'],
    ['France','Mayotte'],
    ['France','Réunion'],
    ['France','Saint Barthelemy'],
    ['France','Saint Pierre and Miquelon'],
    ['Denmark','Faeroe Islands'],
    ['Denmark','Greenland'],
    ['US','American Samoa'],
    ['US','Guam'],
    ['US','Northern Mariana Islands'],
    ['US','Puerto Rico'],
    ['US','U.S. Virgin Islands'],
    ['Israel','State of Palestine'],
    ['United Kingdom','Falkland Islands'],
    ['United Kingdom','Anguilla'],
    ['United Kingdom','Bermuda'],
    ['United Kingdom','British Virgin Islands'],
    ['United Kingdom','Cayman Islands'],
    ['United Kingdom','Channel Islands'],
    ['United Kingdom','Gibraltar'],
    ['United Kingdom','Isle of Man'],
    ['United Kingdom','Montserrat'],
    ['United Kingdom','Saint Helena'],
    ['United Kingdom','Turks and Caicos'],
    ['Netherlands','Aruba'],
    ['Netherlands','Caribbean Netherlands'],
    ['Netherlands','Curaçao'],
    ['Netherlands','Sint Maarten'],
    ['New Zealand','Cook Islands'],
    ['New Zealand','Niue'],
    ['New Zealand','Tokelau'],
    ['China','Hong Kong'],
    ['China','Macao'],
    ['Micronesia','Nauru'],
    ['Micronesia','Palau'],
    ['Morocco','Western Sahara']
]

for change in includeChanges:  
    df_poblacion.index = [change[0] if country == change[1] else country for country in df_poblacion.index]

# Suma población de lugares unidos
df_poblacion = df_poblacion.groupby(df_poblacion.index).sum()

df_poblacion.head(5)

,Population (2020)
Afghanistan,39074280
Albania,2877239
Algeria,43984569
Andorra,77287
Angola,33032075


3. Haga un merge de los DataFrame anteriores. El objetivo es asignar un valor de Population a cada elemento del primer DataFrame. Indique las filas donde no se puede realizar el merge y luego descártelas de su DataFrame final

In [18]:
# Elimina lugares en df de población
removePop = ['Kiribati', 'North Korea', 'Saint Martin', 'Tonga', 'Turkmenistan', 'Tuvalu']
df_poblacion = df_poblacion.drop(removePop,axis=0).sort_index()
df_poblacion.head(5)

,Population (2020)
Afghanistan,39074280
Albania,2877239
Algeria,43984569
Andorra,77287
Angola,33032075


In [19]:
# Elimina lugares en df de confirmados
removeCon = ['Diamond Princess', 'Kosovo', 'MS Zaandam', 'West Bank and Gaza']
df_confirmados = df_confirmados.drop(removeCon,axis=0).sort_index()
df_confirmados.head(5)

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2021-04-16,2021-04-17,2021-04-18,2021-04-19,2021-04-20,2021-04-21,2021-04-22,2021-04-23,2021-04-24,2021-04-25
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,57612,57721,57793,57898,58037,58214,58312,58542,58730,58843
Albania,0,0,0,0,0,0,0,0,0,0,...,129307,129456,129594,129694,129842,129980,130114,130270,130409,130537
Algeria,0,0,0,0,0,0,0,0,0,0,...,119323,119486,119642,119805,119992,120174,120363,120562,120736,120922
Andorra,0,0,0,0,0,0,0,0,0,0,...,12712,12771,12805,12805,12874,12917,12942,13007,13024,13060
Angola,0,0,0,0,0,0,0,0,0,0,...,24122,24300,24389,24518,24661,24883,25051,25279,25492,25609


In [20]:
# Incluye la población al df de confirmados
df_confirmados.insert(0, 'Population', df_poblacion['Population (2020)'])
df_confirmados.head(5)

,Population,2020-01-22 00:00:00,2020-01-23 00:00:00,2020-01-24 00:00:00,2020-01-25 00:00:00,2020-01-26 00:00:00,2020-01-27 00:00:00,2020-01-28 00:00:00,2020-01-29 00:00:00,2020-01-30 00:00:00,...,2021-04-16 00:00:00,2021-04-17 00:00:00,2021-04-18 00:00:00,2021-04-19 00:00:00,2021-04-20 00:00:00,2021-04-21 00:00:00,2021-04-22 00:00:00,2021-04-23 00:00:00,2021-04-24 00:00:00,2021-04-25 00:00:00
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,39074280,0,0,0,0,0,0,0,0,0,...,57612,57721,57793,57898,58037,58214,58312,58542,58730,58843
Albania,2877239,0,0,0,0,0,0,0,0,0,...,129307,129456,129594,129694,129842,129980,130114,130270,130409,130537
Algeria,43984569,0,0,0,0,0,0,0,0,0,...,119323,119486,119642,119805,119992,120174,120363,120562,120736,120922
Andorra,77287,0,0,0,0,0,0,0,0,0,...,12712,12771,12805,12805,12874,12917,12942,13007,13024,13060
Angola,33032075,0,0,0,0,0,0,0,0,0,...,24122,24300,24389,24518,24661,24883,25051,25279,25492,25609


4. Repita el procedimiento anterior para los archivos covid19_recuperados.csv y covid19_muertes.csv

In [21]:
# Importamos datasets
df_recuperados = pd.read_csv('./data/covid19_recuperados.csv',index_col='Country/Region')
df_muertes = pd.read_csv('./data/covid19_decesos.csv',index_col='Country/Region')

# Elimina columnas Lat, Long y Province/State
df_recuperados = df_recuperados.drop(['Lat','Long','Province/State'],axis=1)
df_muertes = df_muertes.drop(['Lat','Long','Province/State'],axis=1)

# Convierte nombre de columnas a datetime64
df_recuperados.columns = pd.to_datetime(df_recuperados.columns).tolist()
df_muertes.columns = pd.to_datetime(df_muertes.columns).tolist()

# Compacta las filas del mismo país en una y suma sus valores
df_recuperados = df_recuperados.groupby("Country/Region").sum()
df_muertes = df_muertes.groupby("Country/Region").sum()

# Elimina Filas Conflictivas
df_recuperados = df_recuperados.drop(removeCon,axis=0).sort_index()
df_muertes = df_muertes.drop(removeCon,axis=0).sort_index()

# Incluye la población
df_recuperados.insert(0, 'Population', df_poblacion['Population (2020)'])
df_muertes.insert(0, 'Population', df_poblacion['Population (2020)'])

In [22]:
# Verifica recuperados
df_recuperados.head(5)

,Population,2020-01-22 00:00:00,2020-01-23 00:00:00,2020-01-24 00:00:00,2020-01-25 00:00:00,2020-01-26 00:00:00,2020-01-27 00:00:00,2020-01-28 00:00:00,2020-01-29 00:00:00,2020-01-30 00:00:00,...,2021-04-16 00:00:00,2021-04-17 00:00:00,2021-04-18 00:00:00,2021-04-19 00:00:00,2021-04-20 00:00:00,2021-04-21 00:00:00,2021-04-22 00:00:00,2021-04-23 00:00:00,2021-04-24 00:00:00,2021-04-25 00:00:00
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,39074280,0,0,0,0,0,0,0,0,0,...,52105,52116,52168,52244,52272,52301,52348,52363,52392,52489
Albania,2877239,0,0,0,0,0,0,0,0,0,...,100600,101142,101584,102171,102601,103066,103582,104278,105016,105728
Algeria,43984569,0,0,0,0,0,0,0,0,0,...,83169,83286,83397,83514,83636,83765,83900,84038,84167,84299
Andorra,77287,0,0,0,0,0,0,0,0,0,...,12105,12159,12203,12203,12285,12334,12375,12423,12458,12491
Angola,33032075,0,0,0,0,0,0,0,0,0,...,22203,22576,22597,22600,22647,22882,22901,23089,23092,23092


In [23]:
# Verifica muertes
df_muertes.head(5)

,Population,2020-01-22 00:00:00,2020-01-23 00:00:00,2020-01-24 00:00:00,2020-01-25 00:00:00,2020-01-26 00:00:00,2020-01-27 00:00:00,2020-01-28 00:00:00,2020-01-29 00:00:00,2020-01-30 00:00:00,...,2021-04-16 00:00:00,2021-04-17 00:00:00,2021-04-18 00:00:00,2021-04-19 00:00:00,2021-04-20 00:00:00,2021-04-21 00:00:00,2021-04-22 00:00:00,2021-04-23 00:00:00,2021-04-24 00:00:00,2021-04-25 00:00:00
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,39074280,0,0,0,0,0,0,0,0,0,...,2535,2539,2539,2546,2549,2557,2561,2565,2572,2582
Albania,2877239,0,0,0,0,0,0,0,0,0,...,2337,2340,2342,2347,2353,2358,2364,2367,2372,2378
Algeria,43984569,0,0,0,0,0,0,0,0,0,...,3148,3152,3155,3160,3165,3172,3181,3190,3198,3207
Andorra,77287,0,0,0,0,0,0,0,0,0,...,123,123,123,123,123,123,123,123,124,124
Angola,33032075,0,0,0,0,0,0,0,0,0,...,560,561,561,563,565,570,572,574,577,579


## Análisis global (1.5pt)
1. Obtenga la cantidad total a la fecha (26 de Abril) de los casos (a) confirmados, (b) cerrados (recuperados más decesos) y (c) activos (confirmados menos cerrados) por país. En base a estas cantidades calcule el porcentaje de
    - casos activos con respecto al total de confirmados
    - decesos con respecto al total de casos cerrados
    - recuperados con respecto al total de casos cerrados

En los archivos csv, varios países reportan información que podría no ser fidedigna.

Por ejemplo, en Suecia ('Sweden') se reportan cero recuperados para todos los días. En otros países, como Estados Unidos ('US'), se reportan cantidades decrecientes de recuperados, lo cual no tiene sentido, ya que los datos supuestamente corresponden a la cantidad acumulada de recuperados. Algo similar ocurre tambien en los archivos de casos confirmados y de muertes

Escribimos un procedimiento que modifica los tres dataframes para que no existan deltas negativos al avanzar en el tiempo.

Para un determinado día, la información después de corregir los dataframes será el valor máximo que se ha alcanzado considerando ese día y todos los anteriores.

In [24]:
# Reemplazamos deltas negativos por el máximo anterior

for row in range(df_confirmados.shape[0]):
    for col in range(2,df_confirmados.shape[1]):
        if df_confirmados.iloc[row,col] < df_confirmados.iloc[row,col-1]:
            df_confirmados.iloc[row,col] = df_confirmados.iloc[row,col-1]
        if df_recuperados.iloc[row,col] < df_recuperados.iloc[row,col-1]:
            df_recuperados.iloc[row,col] = df_recuperados.iloc[row,col-1]
        if df_muertes.iloc[row,col] < df_muertes.iloc[row,col-1]:
            df_muertes.iloc[row,col] = df_muertes.iloc[row,col-1]

In [25]:
# Totales

# Calcula casos confirmados globales hasta el 26 de abril
total_confirmados = df_confirmados[df_confirmados.columns[-1]]

# Calcula casos recuperados globales hasta el 26 de abril
total_recuperados = df_recuperados[df_recuperados.columns[-1]]

# Calcula decesos globales hasta el 26 de abril
total_muertes = df_muertes[df_muertes.columns[-1]]

# Calcula casos cerrados (recuperados más decesos) hasta el 26 de abril
total_cerrados = total_recuperados + total_muertes

# Calcula casos activos (confirmados menos cerrados) hasta el 26 de abril
total_activos = total_confirmados - total_cerrados

# Porcentajes

# Porcentaje de casos activos con respecto al total de confirmados
p_activos_confirmados = 100.0 * total_activos.astype('float64')/total_confirmados.astype('float64')

# Porcentaje de decesos con respecto al total de casos cerrados
p_muertes_cerrados = 100.0 * total_muertes.astype('float64')/total_cerrados.astype('float64')

# Porcentaje de recuperados con respecto al total de casos cerrados
p_recuperados_cerrados = 100.0 * total_recuperados.astype('float64')/total_cerrados.astype('float64')

2. Luego calcule las siguientes tasas en base a información a la fecha
    - Tasa de incidencia: confirmados por 100.000 dividido población total
    - Tasa de recuperación: recuperados por 100.000 dividido población total
    - Tasa de mortalidad: decesos por 100.000 dividido población total

In [26]:
# Tasas

# Población por país (double)
poblacion_total = df_poblacion['Population (2020)'].astype('float64')

# Tasa de incidencia
tasa_incidencia = (100000.0 * total_confirmados)/poblacion_total

# Tasa de recuperación
tasa_recuperacion = (100000.0 * total_recuperados)/poblacion_total

# Tasa de mortalidad
tasa_mortalidad = (100000.0 * total_muertes)/poblacion_total

In [27]:
recuperados_cero = ['US', 'Sweden', 'Serbia', 'Belgium']
recuperados_decrecientes = []

3. En base a lo anterior muestre los nombres y los valores de los 5 países con
    - La mayor y menor tasa de incidencia, respectivamente
    - La mayor y menor tasa de recuperación, respectivamente
    - La mayor y menor tasa de mortalidad, respectivamente

In [28]:
# Ordenamos en orden creciente cada df de tasas
tasa_incidencia_sorted = tasa_incidencia.sort_values()
tasa_recuperacion_sorted = tasa_recuperacion.sort_values()
tasa_mortalidad_sorted = tasa_mortalidad.sort_values()

# Obtenemos 5 mínimos en cada caso
tasa_incidencia_mins = tasa_incidencia_sorted[:5]
tasa_recuperacion_mins = tasa_recuperacion_sorted[:9] # Primeros 4 no son confiables
tasa_mortalidad_mins = tasa_mortalidad_sorted[:5]

# Obtenemos 5 máximos en cada caso
tasa_incidencia_maxs = tasa_incidencia_sorted[-5:]
tasa_recuperacion_maxs = tasa_recuperacion_sorted[-5:]
tasa_mortalidad_maxs = tasa_mortalidad_sorted[-5:]

In [29]:
# Muestra min y max tasas de incidencia
print("Paises con tasas de incidencia mínima:")
display(tasa_incidencia_mins)
print("Paises con tasas de incidencia máxima:")
display(tasa_incidencia_maxs)

Paises con tasas de incidencia mínima:


Country/Region
Micronesia         0.693597
Tanzania           0.848158
Vanuatu            1.297282
Samoa              1.510247
Solomon Islands    2.899933
dtype: float64

Paises con tasas de incidencia máxima:


Country/Region
Slovenia      11401.118524
San Marino    14868.607118
Czechia       15116.787605
Montenegro    15389.759266
Andorra       16898.055300
dtype: float64

In [30]:
# Muestra min y max tasas de recuperacion
print("Paises con tasas de recuperación mínima:")
display(tasa_recuperacion_mins)
print("Paises con tasas de recuperación máxima:")
display(tasa_recuperacion_maxs)

Paises con tasas de recuperación mínima:


Country/Region
Sweden             0.000000
Tanzania           0.304937
Laos               0.671827
Micronesia         0.693597
Vanuatu            0.972961
Samoa              1.006831
Vietnam            2.580777
Solomon Islands    2.609940
Taiwan*            4.386265
dtype: float64

Paises con tasas de recuperación máxima:


Country/Region
Slovenia      10695.487085
Czechia       14227.759190
San Marino    14255.833137
Montenegro    14744.777735
Andorra       16161.838343
dtype: float64

Serbia, Bélgica, Suecia y Estados Unidos reportan cero recupardos, y estimamos que esa información podría no ser verídica.

In [31]:
# Muestra min y max tasas de mortalidad
print("Paises con tasas de mortalidad mínima:")
display(tasa_mortalidad_mins)
print("Paises con tasas de mortalidad máxima:")
display(tasa_mortalidad_maxs)

Paises con tasas de mortalidad mínima:


Country/Region
Laos                     0.0
Holy See                 0.0
Samoa                    0.0
Saint Kitts and Nevis    0.0
Solomon Islands          0.0
dtype: float64

Paises con tasas de mortalidad máxima:


Country/Region
Montenegro                233.250541
Bosnia and Herzegovina    250.308805
San Marino                262.196559
Czechia                   270.357539
Hungary                   275.735780
dtype: float64

4. Responda, analice y discuta: ¿En qué lugar/ranking se encuentra Chile con respecto a cada una de las métricas de los puntos anteriores? ¿A qué países de la OCDE se parece más? ¿A qué paises de latinoamérica se parece más?

In [32]:
paises_ocde = ['Canada', 'United States', 'United Kingdom', 'Denmark', 'Iceland', 'Norway',
               'Turkey','Spain', 'Portugal', 'France', 'Ireland', 'Belgium', 'Germany',
               'Greece', 'Sweden', 'Switzerland','Austria', 'Netherlands', 'Luxembourg',
               'Italy', 'Japan', 'Finland', 'Australia', 'New Zealand', 'Mexico',
               'Czech Republic (Czechia)','Hungary', 'Poland', 'South Korea', 'Slovakia',
               'Chile', 'Slovenia', 'Israel', 'Estonia', 'Latvia', 'Lithuania', 'Colombia']

# tasa_incidencia_sorted_solo_ocde = tasa_incidencia.ix[paises_ocde]


In [33]:
print('Tasa de incidencia en Chile:', tasa_incidencia.at['Chile'])
print('Puesto de Chile en el ranking mundial de incidencia:',
      tasa_incidencia_sorted.index.get_loc('Chile'), '\n')

print('Tasa de recuperación en Chile:', tasa_recuperacion.at['Chile'])
print('Puesto de Chile en el ranking mundial de recuperacion:',
      tasa_recuperacion_sorted.index.get_loc('Chile'), '\n')

print('Tasa de mortalidad en Chile:', tasa_recuperacion.at['Chile'])
print('Puesto de Chile en el ranking mundial de mortalidad:',
      tasa_mortalidad_sorted.index.get_loc('Chile'), '\n')

Tasa de incidencia en Chile: 6108.958633515813
Puesto de Chile en el ranking mundial de incidencia: 147 

Tasa de recuperación en Chile: 5745.315716882119
Puesto de Chile en el ranking mundial de recuperacion: 160 

Tasa de mortalidad en Chile: 5745.315716882119
Puesto de Chile en el ranking mundial de mortalidad: 156 

